In [ ]:
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation, Conv2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import SGD, Adadelta, Adagrad, Adam, Adamax, RMSprop, Nadam
from keras.callbacks import CSVLogger, LearningRateScheduler
from PIL import Image
import numpy as np
import glob
import matplotlib.pyplot as plt
import time
import os, random
import pandas as pd

In [ ]:
n_categories = 2
batch_size = 32
train_dir = './train'
validation_dir = './valid'
test_dir = './tests'
display_dir = './dis'
file_name = 'rate_const_01'
label = ['beer', 'wine']

train_datagen = ImageDataGenerator(rescale=1.0/255)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(56, 56),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

validation_generator=validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(56, 56),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(56, 56, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(n_categories-1))
model.add(Activation('sigmoid'))

model.summary()

In [ ]:
def step_decay(epoch):
    x = 0.1
    if epoch >= 50: x = 0.01
    if epoch >= 75: x = 0.001
    return x
lr_decay = LearningRateScheduler(step_decay)

model.compile(optimizer=SGD(learning_rate=0.1), loss='binary_crossentropy', metrics='accuracy')

In [ ]:
hist = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    validation_data=validation_generator,
    callbacks=[CSVLogger(file_name+'.csv')])

model.save(file_name+'.h5')

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('./', save_format='json')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size=(56, 56),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)


model.compile(optimizer=SGD(lr=0.0001,momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

score=model.evaluate_generator(test_generator)
print('\n test loss:',score[0])
print('\n test_acc:',score[1])

#predict model and display images
files=os.listdir(display_dir)
img=random.sample(files,25)

plt.figure(figsize=(10,10))
for i in range(25):
    temp_img=load_img(os.path.join(display_dir,img[i]),target_size=(224,224))
    plt.subplot(5,5,i+1)
    plt.imshow(temp_img)
    #Images normalization
    temp_img_array=img_to_array(temp_img)
    temp_img_array=temp_img_array.astype('float32')/255.0
    temp_img_array=temp_img_array.reshape((1,224,224,3))
    #predict image
    img_pred=model.predict(temp_img_array)
    plt.title(label[np.argmax(img_pred)])
    #eliminate xticks,yticks
    plt.xticks([]),plt.yticks([])


plt.show()